In [69]:
"""
Name: Michael Austin
Login-ID: maa0075
ID number: 800-259-122
Assignment #: PA-2
Date: 04/06/2019
Purpose: Implement a simplified access control platform. 
"""

import os
import sys
import platform

"""
Constants
"""
START = 'friendadd'
NAME_LEN = 30
NAME_FORB =['/',':', '\n','\t',' ']
CONTENT_FORB = ['friends.txt','lists.txt','pictures.txt','audit.txt']
COMMANDS = ['friendadd','viewby','logout','listadd','friendlist','postpicture',
            'chlst','chmod','chown','readcomments','writecomments','end']

# Output file names 
FRIEND = 'friends.txt'
LIST = 'lists.txt'
PIC = 'pictures.txt'
AUDIT = 'audit.txt'

viewing_profile = ""
friends_list = []
pic_names_list = []
list_names = []
friend_list_assoc = {}

def get_file(filename):
    """
    File not found in cwd, start from the root and work down 
    until you find the file
    """   
    # Detects the OS: (Windows, Linux, MacOS = Darwin)
    this_os = platform.system()
    start = ''
    # Decides which root to start with
    if this_os == 'Windows':
        start = 'c:\\users'
    elif this_os == 'Linux':
        start = 'root\\'
    elif this_os == 'Darwin':
        start = '\\User\\'
    
    cwd = os.getcwd()
    try:
    # Checks current directory for files, first.
        if filename in cwd:
            file = os.path.abspath(cwd+'\\'+filename)
            print(file)
     
        else:
            for root, dirs, files in os.walk(start):
                for name in files:
                    if name == filename:
                        return os.path.abspath(os.path.join(root, name))
    except FileNotFoundError as err:
        make_file(AUDIT, err+' File was not found.')
        print('File was not found')
    
    

def read_text(file):
    """
    Returns each line in the commands file as a list
    """
    line_list = []
    # Standard Python open(), specifying read mode and taking in filename from above
    try:
        with open(file,'r') as f:
            for line in f:
                # 
                line_list.append(line.split())
        return line_list
    except Exception as err:
        add_output_entry(AUDIT, err+' Empty/ Incorrect Command File\n')
        print('Empty/ Incorrect Command File\n' + err)
        


def make_file(filename, first_entry):
    """  Creates the initial friends, audit, list, or pictures output files  """
    with open(filename, 'w') as f:
        f.write(first_entry)


def add_output_entry(file,text):
    with open(file, 'a') as f:
        f.write(text + "\n")

def pre_check(cmd_list):
    for i in cmd_list:
        if i[0] not in COMMANDS:
            add_output_entry(AUDIT,f'{i} is not one of the valid commands.')
            print(f'{i} is not one of the valid commands.')

def get_friend_list(friend_name):
    """ 
    Gets the friends list of a particular user. 
    Input: target person whose list of friends we're looking for. 
    Output: List of friends. 
    """
    for i in friends_list:
        if i == friend_name:
            message = f'Returned friend list for {friend_name}'
            add_output_entry(AUDIT, message)
            print(message)
            return friends_list[i]
        else:
            message = 'Error: Name given does not exist.'
            add_output_entry(AUDIT, message)
            print(message)
            raise KeyError
            
def friend_add(friend_name):
    """
    Reads friends.txt to see if friend name is in there.
    If not, append to the text file, otherwise, throw and log error.
    Input: friend name as a string
    Output: None
    """
    with open(FRIEND, 'r') as file:
        names = file.readlines()
    
    if friend_name not in friends_list:
        if friend_name not in names:
            add_output_entry(FRIEND,friend_name+'\n')
            friends_list.append(friend_name)
            message = f'Added {friend_name} to friends.txt'
            add_output_entry(AUDIT, message)
            print(message)
    else:
        message = 'Error: Name/ friend already in friends list.'
        add_output_entry(AUDIT, message)
        print(message)
        
def friend_list_association(friend_name, list_name):
    """
    Adds a friend to an existing list as long as they are:
    1. A friend of the profile owner; 2. The list name exists; 3. Their name is not currently in the list.
    Input: The friend's name and the list name as strings
    Output: None
    """

    if friend_name in friends_list and list_name in friend_list_assoc and friend_name not in friend_list_assoc[list_name]:
        friend_list_assoc[list_name] += [friend_name]
        message = f'Added {friend_name} to \'{list_name}\' list associations.'
        add_output_entry(AUDIT, message)
        print(message)
    else:
        message = f'Error: {friend_name} not a friend, or {list_name} does not exist.'
        add_output_entry(AUDIT, message)
        print(message)
        
def viewby(name, target_profile):
        """
        Allows only people in friends list to view the profile passed in, then 
        adds them to the viewing list (which can be no larger than 1 element)
        and appends a message to screen and audit.txt.
        Input: name (string) of person trying to view the profile (string) passed in 
        as second parameter
        Output: None. Modifies global viewing list for that profile. 
        """
        global viewing_profile
        # will need to swap out allowed list name below.
        if name in friends_list and not viewing_profile:
            message = f'{name} is viewing {target_profile}\'s profile.'
            add_output_entry(AUDIT, message)
            viewing_profile = name
            print(message)
        else:
            message = f'{name} cannot view {target_profile}.'
            add_output_entry(AUDIT, message)
            print(message)

In [70]:
file = get_file('TestCase1.txt')
f = read_text(file)
make_file(AUDIT, 'Open Command File.\n')
print('Open Command File')
pre_check(f)
make_file(FRIEND, '')
make_file(LIST, '')
make_file(PIC, '')
add_output_entry(AUDIT, 'Initialized output text files.')
print('Initialized output text files.')
#friend_add('Josh')
viewby('Josh', 'me')

Open Command File
Initialized output text files.
Josh cannot view me.


In [9]:
def logout(name):
    if name in who_view_profile:
        who_view_profile.remove(name)
        message = f'{name} has logged out.'
        add_output_entry(AUDIT, message)
        print(message)
        
print(viewing_profile)
logout('James')
print(viewing_profile)

['James']
James has logged out.
[]


In [19]:
def make_person(name, lists=[], friends=[]):
    
    for letter in name:
        if letter in NAME_FORB:
            message = f'Error: {name} cannot contain \'{letter}\'.'
            add_output_entry(AUDIT, message)
            print(message)
        
    return {'name':name, 'lists': lists, "friends":friends}

test = make_person(name='John')
print(test)



Error: Jo hn cannot contain ' '.
{'name': 'Jo hn', 'lists': [], 'friends': []}


In [2]:

"""
def main(arg):

    
    Gets the root of the current operating system for the absolute filepath
    Returns a file object
    
    file = get_file(arg)
    
    
    Each element in f is a list in which each word in the line of 
    text read in is an element. E.g. f = [[friendadd,root], [viewby, root], ...]
    
    f = read_text(file)
    #print(f[0])
    make_file(AUDIT, 'Open Command File.\n')
    print('Open Command File')
    pre_check(f)
    for command in f:
        if command[0] == "friendadd":
            pass
        elif command[0] == "viewby":
        
        elif command[0] == "listadd":
        
        elif command[0] == "friendlist":
        
        elif command[0] == "postpicture":
        
        elif command[0] == "chlst":
        
        elif command[0] == "chmod":
        
        elif command[0] == "chown":
        
        elif command[0] == "readcomments":
        
        elif command[0] == "writecomments":
        
        elif command[0] == "logout":
        
        elif command[0] == "end":
        
        else:
            pass

    

#main('TestCase1.txt')

if __name__=="__main__":
    main(sys.argv[1])
"""
    

'\ndef main(arg):\n\n    \n    Gets the root of the current operating system for the absolute filepath\n    Returns a file object\n    \n    file = get_file(arg)\n    \n    \n    Each element in f is a list in which each word in the line of \n    text read in is an element. E.g. f = [[friendadd,root], [viewby, root], ...]\n    \n    f = read_text(file)\n    #print(f[0])\n    make_file(AUDIT, \'Open Command File.\n\')\n    print(\'Open Command File\')\n    pre_check(f)\n    for command in f:\n        if command[0] == "friendadd":\n            pass\n        elif command[0] == "viewby":\n        \n        elif command[0] == "listadd":\n        \n        elif command[0] == "friendlist":\n        \n        elif command[0] == "postpicture":\n        \n        elif command[0] == "chlst":\n        \n        elif command[0] == "chmod":\n        \n        elif command[0] == "chown":\n        \n        elif command[0] == "readcomments":\n        \n        elif command[0] == "writecomments":\n    